IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M2\FAULT_M2_5


In [3]:
guasto = 1

VOLO m2 FAULT 5%

In [4]:
rcou_m2_fault5 = pd.read_csv("RCOU.csv")
rcou_m2_fault5 = rcou_m2_fault5.astype("float64")
rcou_m2_fault5 = rcou_m2_fault5.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m2_fault5 = rcou_m2_fault5[((rcou_m2_fault5['C9'] >= potenza) & (rcou_m2_fault5['C10'] >= potenza) & (rcou_m2_fault5['C11'] >= potenza) & (rcou_m2_fault5['C12'] >= potenza) & (rcou_m2_fault5['C13'] >= potenza) & (rcou_m2_fault5['C14']>= potenza))]
rcou_m2_fault5=rcou_m2_fault5.reset_index(drop=True)
display(rcou_m2_fault5)

,TimeUS,C9,C10,C11,C12,C13,C14
0,70497686.0,1379.0,1389.0,1404.0,1363.0,1397.0,1371.0
1,70508601.0,1354.0,1424.0,1398.0,1383.0,1359.0,1420.0
2,70519703.0,1368.0,1423.0,1384.0,1408.0,1352.0,1438.0
3,70530500.0,1403.0,1402.0,1413.0,1392.0,1411.0,1394.0
4,70561170.0,1480.0,1352.0,1412.0,1428.0,1469.0,1366.0
...,...,...,...,...,...,...,...
5038,131358716.0,1506.0,1466.0,1525.0,1446.0,1443.0,1527.0
5039,131368599.0,1519.0,1450.0,1526.0,1441.0,1457.0,1513.0
5040,131379386.0,1541.0,1415.0,1523.0,1437.0,1480.0,1484.0
5041,131396916.0,1503.0,1435.0,1545.0,1384.0,1472.0,1468.0


In [5]:
min=rcou_m2_fault5['TimeUS'][0]
max=rcou_m2_fault5['TimeUS'][len(rcou_m2_fault5)-1]
print(max)
print(min)

131407710.0
70497686.0


In [6]:
xkf1_m2_fault5 = pd.read_csv("XKF1_0.csv")
xkf1_m2_fault5 = xkf1_m2_fault5.astype("float64")
xkf1_m2_fault5 = xkf1_m2_fault5.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m2_fault5 = xkf1_m2_fault5[((xkf1_m2_fault5['TimeUS'] >= min) & (xkf1_m2_fault5['TimeUS'] <= max))]
xkf1_m2_fault5 = xkf1_m2_fault5.reset_index(drop=True)
print(xkf1_m2_fault5)

           TimeUS  Roll  Pitch    Yaw
0      70506077.0  0.86  -0.80  22.99
1      70516653.0  0.82  -0.78  23.02
2      70527788.0  0.81  -0.82  23.02
3      70538645.0  0.87  -0.88  23.03
4      70548617.0  0.89  -0.88  23.02
...           ...   ...    ...    ...
5534  131356703.0 -1.94   2.88  22.24
5535  131366708.0 -1.93   2.76  22.34
5536  131376923.0 -1.88   2.59  22.41
5537  131394548.0 -1.80   2.27  22.51
5538  131405296.0 -1.77   2.09  22.55

[5539 rows x 4 columns]


In [7]:
att_m2_fault5 = pd.read_csv("ATT.csv")
att_m2_fault5 = att_m2_fault5.astype("float64")
att_m2_fault5 = att_m2_fault5.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m2_fault5 = att_m2_fault5[((att_m2_fault5['TimeUS'] >= min) & (att_m2_fault5['TimeUS'] <= max	))]
att_m2_fault5=att_m2_fault5.reset_index(drop=True)
print(att_m2_fault5)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      70507692.0     0.00  0.86      0.06  -0.80   22.93  22.99
1      70518797.0     0.00  0.82      0.06  -0.78   22.93  23.02
2      70529634.0     0.00  0.81      0.06  -0.82   22.93  23.02
3      70539879.0     0.01  0.87      0.05  -0.88   22.93  23.03
4      70549895.0     0.01  0.89      0.05  -0.88   22.93  23.02
...           ...      ...   ...       ...    ...     ...    ...
5537  131358142.0     1.94 -1.94      0.46   2.88   25.39  22.24
5538  131367962.0     2.01 -1.93      0.41   2.76   25.39  22.34
5539  131378544.0     2.06 -1.88      0.37   2.59   25.38  22.41
5540  131396170.0     2.10 -1.80      0.34   2.27   25.38  22.51
5541  131407152.0     2.11 -1.77      0.33   2.09   25.38  22.55

[5542 rows x 7 columns]


In [8]:
if len(att_m2_fault5)>len(xkf1_m2_fault5):
    to_add=att_m2_fault5[len(xkf1_m2_fault5):]
    att_m2_fault5=att_m2_fault5[:len(xkf1_m2_fault5)]

if len(xkf1_m2_fault5)>len(att_m2_fault5):
    to_add=xkf1_m2_fault5[len(att_m2_fault5):]

for idx,i in enumerate(att_m2_fault5['Roll']):
    if(xkf1_m2_fault5['Roll'][idx] != i):
        att_m2_fault5['Roll'][idx] = ((att_m2_fault5['Roll'][idx] + xkf1_m2_fault5['Roll'][idx])/2)

for idx,i in enumerate(att_m2_fault5['Pitch']):
    if(xkf1_m2_fault5['Pitch'][idx] != i):
        att_m2_fault5['Pitch'][idx] = ((att_m2_fault5['Pitch'][idx] + xkf1_m2_fault5['Pitch'][idx])/2)

for idx,i in enumerate(att_m2_fault5['Yaw']):
    if(xkf1_m2_fault5['Yaw'][idx] != i):
        att_m2_fault5['Yaw'][idx] = ((att_m2_fault5['Yaw'][idx] + xkf1_m2_fault5['Yaw'][idx])/2)

att_m2_fault5=pd.concat([att_m2_fault5,to_add])

print(att_m2_fault5)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw     Yaw
0      70507692.0     0.00  0.86      0.06 -0.800   22.93  22.990
1      70518797.0     0.00  0.82      0.06 -0.780   22.93  23.020
2      70529634.0     0.00  0.81      0.06 -0.820   22.93  23.020
3      70539879.0     0.01  0.87      0.05 -0.880   22.93  23.030
4      70549895.0     0.01  0.89      0.05 -0.880   22.93  23.020
...           ...      ...   ...       ...    ...     ...     ...
5537  131358142.0     1.94 -1.87      0.46  2.575   25.39  22.375
5538  131367962.0     2.01 -1.85      0.41  2.425   25.39  22.445
5539  131378544.0     2.06 -1.88      0.37  2.590   25.38  22.410
5540  131396170.0     2.10 -1.80      0.34  2.270   25.38  22.510
5541  131407152.0     2.11 -1.77      0.33  2.090   25.38  22.550

[5542 rows x 7 columns]


In [9]:
'''#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m2_fault5 = pd.read_csv("ESC_0.csv")
esc_0_m2_fault5 = esc_0_m2_fault5.astype("float64")
esc_0_m2_fault5 = esc_0_m2_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m2_fault5 = esc_0_m2_fault5[((esc_0_m2_fault5['TimeUS'] >= min) & (esc_0_m2_fault5['TimeUS'] <= max))]
esc_0_m2_fault5=esc_0_m2_fault5.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m2_fault5 = pd.read_csv("ESC_1.csv")
esc_1_m2_fault5 = esc_1_m2_fault5.astype("float64")
esc_1_m2_fault5 = esc_1_m2_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m2_fault5 = esc_1_m2_fault5[((esc_1_m2_fault5['TimeUS'] >= min) & (esc_1_m2_fault5['TimeUS'] <= max))]
esc_1_m2_fault5=esc_1_m2_fault5.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m2_fault5 = pd.read_csv("ESC_2.csv")
esc_2_m2_fault5 = esc_2_m2_fault5.astype("float64")
esc_2_m2_fault5 = esc_2_m2_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m2_fault5 = esc_2_m2_fault5[((esc_2_m2_fault5['TimeUS'] >= min) & (esc_2_m2_fault5['TimeUS'] <= max))]
esc_2_m2_fault5=esc_2_m2_fault5.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m2_fault5 = pd.read_csv("ESC_3.csv")
esc_3_m2_fault5 = esc_3_m2_fault5.astype("float64")
esc_3_m2_fault5 = esc_3_m2_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m2_fault5 = esc_3_m2_fault5[((esc_3_m2_fault5['TimeUS'] >= min) & (esc_3_m2_fault5['TimeUS'] <= max))]
esc_3_m2_fault5=esc_3_m2_fault5.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m2_fault5 = pd.read_csv("ESC_4.csv")
esc_4_m2_fault5 = esc_4_m2_fault5.astype("float64")
esc_4_m2_fault5 = esc_4_m2_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m2_fault5 = esc_4_m2_fault5[((esc_4_m2_fault5['TimeUS'] >= min) & (esc_4_m2_fault5['TimeUS'] <= max))]
esc_4_m2_fault5=esc_4_m2_fault5.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m2_fault5 = pd.read_csv("ESC_5.csv")
esc_5_m2_fault5 = esc_5_m2_fault5.astype("float64")
esc_5_m2_fault5 = esc_5_m2_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m2_fault5 = esc_5_m2_fault5[((esc_5_m2_fault5['TimeUS'] >= min) & (esc_5_m2_fault5['TimeUS'] <= max))]
esc_5_m2_fault5=esc_5_m2_fault5.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})'''

'#elettricita\' che scorre nel motore considerato, al momento ne consideriamo solo 1\nesc_0_m2_fault5 = pd.read_csv("ESC_0.csv")\nesc_0_m2_fault5 = esc_0_m2_fault5.astype("float64")\nesc_0_m2_fault5 = esc_0_m2_fault5.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_0_m2_fault5 = esc_0_m2_fault5[((esc_0_m2_fault5[\'TimeUS\'] >= min) & (esc_0_m2_fault5[\'TimeUS\'] <= max))]\nesc_0_m2_fault5=esc_0_m2_fault5.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})\n\nesc_1_m2_fault5 = pd.read_csv("ESC_1.csv")\nesc_1_m2_fault5 = esc_1_m2_fault5.astype("float64")\nesc_1_m2_fault5 = esc_1_m2_fault5.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_1_m2_fault5 = esc_1_m2_fault5[((esc_1_m2_fault5[\'TimeUS\'] >= min) & (esc_1_m2_fault5[\'TimeUS\'] <= max))]\nesc_1_m2_fault5=esc_1_m2_fault5.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})\n\nesc_2_m2_fault5 = pd.r

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m2_fault5 = pd.read_csv("IMU_0.csv")
imu_0_m2_fault5 = imu_0_m2_fault5.astype("float64")
imu_0_m2_fault5 = imu_0_m2_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m2_fault5 = imu_0_m2_fault5[((imu_0_m2_fault5['TimeUS'] >= min) & (imu_0_m2_fault5['TimeUS'] <= max))]

imu_1_m2_fault5 = pd.read_csv("IMU_1.csv")
imu_1_m2_fault5 = imu_1_m2_fault5.astype("float64")
imu_1_m2_fault5 = imu_1_m2_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m2_fault5 = imu_1_m2_fault5[((imu_1_m2_fault5['TimeUS'] >= min) & (imu_1_m2_fault5['TimeUS'] <= max))]

imu_2_m2_fault5 = pd.read_csv("IMU_2.csv")
imu_2_m2_fault5 = imu_2_m2_fault5.astype("float64")
imu_2_m2_fault5 = imu_2_m2_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m2_fault5 = imu_2_m2_fault5[((imu_2_m2_fault5['TimeUS'] >= min) & (imu_2_m2_fault5['TimeUS'] <= max))]

imu_m2_fault5 = pd.concat((imu_0_m2_fault5, imu_1_m2_fault5, imu_2_m2_fault5))
imu_m2_fault5=imu_m2_fault5.groupby(imu_m2_fault5.TimeUS, as_index=False).mean()

display(imu_m2_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,70499925.0,0.017612,-0.049395,0.006887,-0.126758,0.039685,-9.754690
1,70501976.0,0.112567,-0.056947,-0.002161,-0.219659,-0.037732,-10.140273
2,70504650.0,0.057938,-0.035145,0.015399,-0.271173,-0.141826,-10.224187
3,70507590.0,-0.042021,0.008419,0.032312,-0.280594,-0.294746,-9.969529
4,70510441.0,-0.043291,-0.072291,0.021327,-0.184740,-0.245387,-9.670237
...,...,...,...,...,...,...,...
21031,131395945.0,0.058947,-0.343453,0.085216,0.791422,1.257212,-12.164007
21032,131399001.0,0.059801,-0.340156,0.073685,0.771485,1.439409,-12.753277
21033,131402115.0,0.108500,-0.310292,0.066445,0.849265,1.520438,-13.021527
21034,131404186.0,0.137428,-0.350096,0.051299,0.959155,1.500244,-13.369807


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m2_fault5 = pd.merge_ordered(imu_m2_fault5,att_m2_fault5)
m2_fault5=m2_fault5.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
'''m2_fault5 = pd.merge_ordered(m2_fault5, esc_0_m2_fault5)
m2_fault5 = pd.merge_ordered(m2_fault5, esc_1_m2_fault5)
m2_fault5 = pd.merge_ordered(m2_fault5, esc_2_m2_fault5)
m2_fault5 = pd.merge_ordered(m2_fault5, esc_3_m2_fault5)
m2_fault5 = pd.merge_ordered(m2_fault5, esc_4_m2_fault5)
m2_fault5 = pd.merge_ordered(m2_fault5, esc_5_m2_fault5)
m2_fault5=m2_fault5.fillna(method="ffill").fillna(method="bfill")'''
m2_fault5 = pd.merge_ordered(m2_fault5, rcou_m2_fault5)
m2_fault5=m2_fault5.fillna(method="ffill").fillna(method="bfill")

In [12]:
m2_fault5["TimeUS"] = m2_fault5["TimeUS"] - m2_fault5.iloc[0]["TimeUS"]
m2_fault5["TimeUS"] = m2_fault5["TimeUS"].astype(int)
m2_fault5["Guasto"] = guasto

In [13]:
last_cell = m2_fault5.iloc[-1, m2_fault5.columns.get_loc('TimeUS')]
print(last_cell)

60910024


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
21315  60896955
21316  60899812
21317  60902669
21318  60905526
21319  60908383

[21320 rows x 1 columns]


In [16]:
m2_fault5_final = pd.merge_ordered(m2_fault5,df)
m2_fault5_final=m2_fault5_final.fillna(method="ffill").fillna(method="bfill")
print(m2_fault5_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0  0.017612 -0.049395  0.006887 -0.126758  0.039685  -9.754690   
1          2239  0.017612 -0.049395  0.006887 -0.126758  0.039685  -9.754690   
2          2857  0.017612 -0.049395  0.006887 -0.126758  0.039685  -9.754690   
3          4290  0.112567 -0.056947 -0.002161 -0.219659 -0.037732 -10.140273   
4          5714  0.112567 -0.056947 -0.002161 -0.219659 -0.037732 -10.140273   
...         ...       ...       ...       ...       ...       ...        ...   
52928  60906500  0.137428 -0.350096  0.051299  0.959155  1.500244 -13.369807   
52929  60908383  0.137428 -0.350096  0.051299  0.959155  1.500244 -13.369807   
52930  60909108  0.068426 -0.334408  0.039655  0.902024  1.485484 -13.588687   
52931  60909466  0.068426 -0.334408  0.039655  0.902024  1.485484 -13.588687   
52932  60910024  0.068426 -0.334408  0.039655  0.902024  1.485484 -13.588687   

       DesRoll  Roll  DesPitch  Pitch  

In [17]:
m2_fault5_final = m2_fault5_final[m2_fault5_final.TimeUS.isin(to_be)]

In [18]:
m2_fault5_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,0,0.017612,-0.049395,0.006887,-0.126758,0.039685,-9.754690,0.00,0.86,0.06,-0.80,22.93,22.99,1379.0,1389.0,1404.0,1363.0,1397.0,1371.0,1.0
2,2857,0.017612,-0.049395,0.006887,-0.126758,0.039685,-9.754690,0.00,0.86,0.06,-0.80,22.93,22.99,1379.0,1389.0,1404.0,1363.0,1397.0,1371.0,1.0
4,5714,0.112567,-0.056947,-0.002161,-0.219659,-0.037732,-10.140273,0.00,0.86,0.06,-0.80,22.93,22.99,1379.0,1389.0,1404.0,1363.0,1397.0,1371.0,1.0
6,8571,0.057938,-0.035145,0.015399,-0.271173,-0.141826,-10.224187,0.00,0.86,0.06,-0.80,22.93,22.99,1379.0,1389.0,1404.0,1363.0,1397.0,1371.0,1.0
10,11428,-0.042021,0.008419,0.032312,-0.280594,-0.294746,-9.969529,0.00,0.86,0.06,-0.80,22.93,22.99,1354.0,1424.0,1398.0,1383.0,1359.0,1420.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52919,60896955,0.076939,-0.319314,0.094927,0.455154,1.119906,-11.507130,2.06,-1.88,0.37,2.59,25.38,22.41,1541.0,1415.0,1523.0,1437.0,1480.0,1484.0,1.0
52923,60899812,0.058947,-0.343453,0.085216,0.791422,1.257212,-12.164007,2.10,-1.80,0.34,2.27,25.38,22.51,1503.0,1435.0,1545.0,1384.0,1472.0,1468.0,1.0
52925,60902669,0.059801,-0.340156,0.073685,0.771485,1.439409,-12.753277,2.10,-1.80,0.34,2.27,25.38,22.51,1503.0,1435.0,1545.0,1384.0,1472.0,1468.0,1.0
52927,60905526,0.108500,-0.310292,0.066445,0.849265,1.520438,-13.021527,2.10,-1.80,0.34,2.27,25.38,22.51,1503.0,1435.0,1545.0,1384.0,1472.0,1468.0,1.0


In [19]:
from datetime import timedelta
m2_fault5_final=m2_fault5_final.reset_index(drop=True)
m2_fault5_final['TimeUS'] = pd.to_datetime(m2_fault5_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m2_fault5_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.017612,-0.049395,0.006887,-0.126758,0.039685,-9.754690,0.00,0.86,0.06,-0.80,22.93,22.99,1379.0,1389.0,1404.0,1363.0,1397.0,1371.0,1.0
1,00:00:00.002857,0.017612,-0.049395,0.006887,-0.126758,0.039685,-9.754690,0.00,0.86,0.06,-0.80,22.93,22.99,1379.0,1389.0,1404.0,1363.0,1397.0,1371.0,1.0
2,00:00:00.005714,0.112567,-0.056947,-0.002161,-0.219659,-0.037732,-10.140273,0.00,0.86,0.06,-0.80,22.93,22.99,1379.0,1389.0,1404.0,1363.0,1397.0,1371.0,1.0
3,00:00:00.008571,0.057938,-0.035145,0.015399,-0.271173,-0.141826,-10.224187,0.00,0.86,0.06,-0.80,22.93,22.99,1379.0,1389.0,1404.0,1363.0,1397.0,1371.0,1.0
4,00:00:00.011428,-0.042021,0.008419,0.032312,-0.280594,-0.294746,-9.969529,0.00,0.86,0.06,-0.80,22.93,22.99,1354.0,1424.0,1398.0,1383.0,1359.0,1420.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21315,00:01:00.896955,0.076939,-0.319314,0.094927,0.455154,1.119906,-11.507130,2.06,-1.88,0.37,2.59,25.38,22.41,1541.0,1415.0,1523.0,1437.0,1480.0,1484.0,1.0
21316,00:01:00.899812,0.058947,-0.343453,0.085216,0.791422,1.257212,-12.164007,2.10,-1.80,0.34,2.27,25.38,22.51,1503.0,1435.0,1545.0,1384.0,1472.0,1468.0,1.0
21317,00:01:00.902669,0.059801,-0.340156,0.073685,0.771485,1.439409,-12.753277,2.10,-1.80,0.34,2.27,25.38,22.51,1503.0,1435.0,1545.0,1384.0,1472.0,1468.0,1.0
21318,00:01:00.905526,0.108500,-0.310292,0.066445,0.849265,1.520438,-13.021527,2.10,-1.80,0.34,2.27,25.38,22.51,1503.0,1435.0,1545.0,1384.0,1472.0,1468.0,1.0


## Analisi nel tempo e in frequenza

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m2_fault5_final)/350)):
        V1=m2_fault5_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
'''rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')'''
c_9 =time_freq_feat('C9')
c_10 =time_freq_feat('C10')
c_11 =time_freq_feat('C11')
c_12 =time_freq_feat('C12')
c_13 =time_freq_feat('C13')
c_14 =time_freq_feat('C14')
#merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5,c_9,c_10,c_11,c_12,c_13,c_14]
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,c_9,c_10,c_11,c_12,c_13,c_14]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_19300\3767042834.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_19300\3767042834.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY),<>(GyrZ),var(GyrZ),kurt(GyrZ),RMS(GyrZ),freq1(GyrZ),freq2(GyrZ),amp1(GyrZ),amp2(GyrZ),<>(AccX),var(AccX),kurt(AccX),RMS(AccX),freq1(AccX),freq2(AccX),amp1(AccX),amp2(AccX),<>(AccY),var(AccY),kurt(AccY),RMS(AccY),freq1(AccY),freq2(AccY),amp1(AccY),amp2(AccY),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ),<>(Roll),var(Roll),kurt(Roll),RMS(Roll),freq1(Roll),freq2(Roll),amp1(Roll),amp2(Roll),<>(Pitch),var(Pitch),kurt(Pitch),RMS(Pitch),freq1(Pitch),freq2(Pitch),amp1(Pitch),amp2(Pitch),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw),<>(C9),var(C9),kurt(C9),RMS(C9),freq1(C9),freq2(C9),amp1(C9),amp2(C9),<>(C10),var(C10),kurt(C10),RMS(C10),freq1(C10),freq2(C10),amp1(C10),amp2(C10),<>(C11),var(C11),kurt(C11),RMS(C11),freq1(C11),freq2(C11),amp1(C11),amp2(C11),<>(C12),var(C12),kurt(C12),RMS(C12),freq1(C12),freq2(C12),amp1(C12),amp2(C12),<>(C13),var(C13),kurt(C13),RMS(C13),freq1(C13),freq2(C13),amp1(C13),amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,0.0,-0.011376,0.005900,-0.249248,0.077541,64.0,87.0,2.605035,2.078838,0.043152,0.013480,-0.996907,0.123707,99.0,98.0,10.018649,2.688976,0.123641,0.008886,-1.459422,0.155396,65.0,51.0,1.317705,0.590070,0.102490,0.096495,-1.083771,0.326687,107.0,105.0,6.629072,5.889159,0.178059,0.047044,0.017452,0.280383,118.0,119.0,13.043662,9.894070,-10.603860,0.610498,-0.866009,10.632526,0.0,117.0,175.384407,9.286425,0.800228,0.093164,-0.750021,0.856308,85.0,82.0,1.965432,1.574238,0.352906,2.036413,-1.725140,1.468044,107.0,105.0,5.869423,0.666816,25.354872,6.192937,-1.249906,25.476358,111.0,84.0,2.955744,-1.102542,1545.495726,5043.113553,0.574115,1547.121775,88.0,90.0,585.325452,530.027715,1474.521368,2573.027399,-0.870792,1475.391121,98.0,97.0,853.556155,472.109674,1542.578348,2960.193130,1.138849,1543.534813,115.0,113.0,832.417909,570.805530,1479.555556,2499.001905,-0.819853,1480.397421,111.0,104.0,718.303571,465.618615,1505.541311,2990.026146,0.300444,1506.531163,104.0,102.0,1754.513037,1372.374659,1518.034188,2564.953114,0.627521,1518.876375,101.0,99.0,690.777746,533.670948,1
1,1.0,-0.011785,0.005942,-0.266223,0.077873,97.0,73.0,2.058438,1.692741,0.043162,0.013441,-0.991306,0.123556,98.0,97.0,10.314474,2.612359,0.123590,0.008862,-1.455545,0.155278,81.0,67.0,1.376636,0.502038,0.102668,0.096231,-1.078646,0.326341,117.0,114.0,8.134907,6.636764,0.179260,0.047418,0.008514,0.281811,114.0,115.0,10.158143,8.581960,-10.601587,0.610577,-0.861247,10.630263,0.0,121.0,175.741466,9.427049,0.798722,0.093697,-0.746150,0.855212,76.0,73.0,2.014703,1.470141,0.356534,2.035244,-1.727111,1.468530,127.0,125.0,6.323447,0.613229,25.368210,6.237919,-1.256591,25.490514,109.0,81.0,2.909665,-2.167478,1545.599432,5032.531396,0.587147,1547.221974,97.0,99.0,522.947469,467.197905,1474.411932,2569.912450,-0.869787,1475.280704,100.0,99.0,1035.810767,580.115457,1542.735795,2960.485553,1.145755,1543.692265,113.0,111.0,838.276980,576.808694,1479.372159,2503.721502,-0.826966,1480.215725,113.0,106.0,721.169239,414.763078,1505.488636,2982.484201,0.305113,1506.476035,96.0,94.0,1694.825854,1297.812353,1518.090909,2558.778037,0.639962,1518.931044,99.0,97.0,825.293672,563.450108,1
2,2.0,-0.012010,0.005943,-0.274531,0.077914,68.0,89.0,3.202913,2.098878,0.043051,0.013408,-0.985069,0.123382,107.0,106.0,10.564421,2.483438,0.123497,0.008840,-1.452600,0.155133,71.0,70.0,1.422168,0.433394,0.103498,0.096201,-1.080451,0.326559,115.0,112.0,9.529467,7.146838,0.180281,0.047651,-0.002957,0.282873,107.0,109.0,8.735084,7.858841,-10.599274,0.610730,-0.856566,10.627964,0.0,113.0,176.102822,9.983467,0.797224,0.094223,-0.742659,0.854120,101.0,98.0,2.072609,1.381973,0.360142,2.034057,-1.728996,1.469012,125.0,123.0,6.765923,0.581809,25.381473,6.282292,-1.263422,25.504581,107.0,

In [23]:
path_file = path_file.replace(r"M2\FAULT_M2_5", "")
os.chdir(path_file)
df_merged.to_csv('m2_fault5.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
